In [172]:
function old(x::Int)
    print("Old behavior")
end

function new(x::Int)
    print("New behavior")
end

# Adding true/false argument doesn't change my observations.
@deprecate old(x::Int) new(x::Int) false

old (generic function with 2 methods)

In [173]:
old(2)

New behavior

In [ ]:
function my_trembling_hand_v3!(A, x, w::Float64) 
    
    shape_A = size(A)
    δ = @SVector [1.0./(shape_A[1+i]-1) for i = 1:(length(shape_A)-1)]


    for n in 1:shape_A[1]

        xn = x[n]
        xn = min.(max.(xn, 0.0),1.0)
        qn = div.(xn, δ)
        qn = max.(0, qn)
        qn = min.(qn, shape_A[2:].-2)
        λn = (xn./δ.-qn) # ∈[0,1[ by construction
        qn_ = round.(Int,qn) + 1

            q0_, q1_ = qn_[1], qn_[2]
            λ0, λ1  = λn[1], λn[2]
            A[n, q0_ ,  q1_] += (1-λ0)*(1-λ1)*w
            A[n, q0_+1, q1_] += λ0*(1-λ1)*w
            A[n, q0_, q1_+1] += (1-λ0)*λ1*w
            A[n, q0_+1, q1_+1] += λ0*λ1*w
        
    end

end

In [41]:
function trembling_hand!(A::AbstractArray{Float64,2}, x, w)
    N,n0 = size(A)
    δ0 = 1.0./(n0-1.0)
    for n in 1:N
        x0 = x[n][1]
        x0 = min.(max.(x0, 0.0),1.0)
        q0 = div.(x0, δ0)
        q0 = max.(0, q0)
        q0 = min.(q0, n0-2)
        λ0 = (x0./δ0-q0) # ∈[0,1[ by construction
        q0_ = round.(Int,q0) + 1
        A[n, q0_]   += (1-λ0)*w
        A[n, q0_+1] += λ0*w
    end
end

trembling_hand! (generic function with 1 method)

In [38]:
using StaticArrays
#using Dolo
A = Array{Float64,2}(undef, 2, 3)
print(A)
x = [SVector{2,Float64}(3.4,4.7),SVector{2,Float64}(2.6,4.2)]
w =1
Dolo.my_trembling_hand_v1!(A,x,w)

MethodError: MethodError: no method matching my_trembling_hand_v1!(::Matrix{Float64}, ::Vector{SVector{2, Float64}}, ::Int64)
Closest candidates are:
  my_trembling_hand_v1!(::Any, ::Array{SVector{d, Float64}, 1}, !Matched::Float64) where d at /Users/gabriellequeran/.julia/dev/Dolo/src/algos/ergodic.jl:164

In [71]:
function my_trembling_hand_v1!(A, x::Vector{Point{d}}, w::Float64) where d

    if d==1
        N,n0 = size(A)
        δ0 = 1.0./(n0-1.0)
        for n in 1:N
            x0 = x[n][1]
            x0 = min.(max.(x0, 0.0),1.0)
            q0 = div.(x0, δ0)
            q0 = max.(0, q0)
            q0 = min.(q0, n0-2)
            λ0 = (x0./δ0-q0) # ∈[0,1[ by construction
            q0_ = round.(Int,q0) + 1
            A[n, q0_]   += (1-λ0)*w
            A[n, q0_+1] += λ0*w
        end
    end

    if d==2
        N,n0,n1 = size(A)
        δ0 = 1.0./(n0-1.0)
        δ1 = 1.0./(n1-1.0)
        for n in 1:N
            x0 = x[n][1]
            x0 = min.(max.(x0, 0.0),1.0)
            q0 = div.(x0, δ0)
            q0 = max.(0, q0)
            q0 = min.(q0, n0-2)
            λ0 = (x0./δ0-q0) # ∈[0,1[ by construction
            q0_ = round.(Int,q0) + 1

            x1 = x[n][2]
            x1 = min.(max.(x1, 0.0),1.0)
            q1 = div.(x1, δ1)
            q1 = max.(0, q1)
            q1 = min.(q1, n1-2)
            λ1 = (x1./δ1-q1) # ∈[0,1[ by construction
            q1_ = round.(Int,q1) + 1

            A[n, q0_ ,  q1_] += (1-λ0)*(1-λ1)*w
            A[n, q0_+1, q1_] += λ0*(1-λ1)*w
            A[n, q0_, q1_+1] += (1-λ0)*λ1*w
            A[n, q0_+1, q1_+1] += λ0*λ1*w
        end
    end

end

UndefVarError: UndefVarError: Point not defined

In [64]:
function my_trembling_hand_v2!(A, x::Vector{Point{d}}, w::Float64) where d

    if d==1
        N,n0 = size(A)
        δ0 = 1.0./(n0-1.0)
        for n in 1:N
            x0 = x[n][1]
            x0 = min.(max.(x0, 0.0),1.0)
            q0 = div.(x0, δ0)
            q0 = max.(0, q0)
            q0 = min.(q0, n0-2)
            λ0 = (x0./δ0-q0) # ∈[0,1[ by construction
            q0_ = round.(Int,q0) + 1
            A[n, q0_]   += (1-λ0)*w
            A[n, q0_+1] += λ0*w
        end
    end

    if d==2
        N,n0,n1 = size(A)
        δ = SVector{2,Float64}(1.0./(n0-1.0), 1.0./(n1-1.0))
        for n in 1:N

            xn = x[n]
            xn = min.(max.(xn, 0.0),1.0)
            qn = div.(xn, δ)
            qn = max.(0, qn)
            qn = min.(qn, n0-2)
            λn = (xn./δ.-qn) # ∈[0,1[ by construction
            qn_ = round.(Int,q0) + 1

            q0_, q1_ = qn_[1], qn_[2]
            λ0, λ1  = λn[1], λn[2]
            A[n, q0_ ,  q1_] += (1-λ0)*(1-λ1)*w
            A[n, q0_+1, q1_] += λ0*(1-λ1)*w
            A[n, q0_, q1_+1] += (1-λ0)*λ1*w
            A[n, q0_+1, q1_+1] += λ0*λ1*w
        end
    end

end

UndefVarError: UndefVarError: Point not defined

In [79]:
exemple = size(A)
length(exemple)

2